In [1]:
import json
import math
from dataclasses import dataclass, field, replace
from itertools import accumulate
from functools import cache
from collections import defaultdict
from heapdict import heapdict
from frozendict import frozendict

with open('save.json') as f:
    save = json.load(f)
    
profile = save['profiles']['def']
profile_relics = profile['relics']
resources = profile['resources']
leafscenders = profile['leafscenders']

In [2]:
rarity_to_level = dict(
    common=1,
    uncommon=2,
    rare=3,
    epic=4,
    mythical=5,
    legendary=6
)

increased_multiplier = { 'celestial', 'obsidian', 'benitoite', 'ancient', 'amber', 'amethyst', 'emerald', 'kyanite', 'rhodonite', 'ruby', 'tektite' }

relics = []
for key, value in profile_relics.items():
    rarity, kind = key.split('_')
    level = rarity_to_level[rarity]
    relics += [dict(level=level, kind=kind, relics=value['count'], ascensions=leafscenders[str(level)][kind]['completed_count'])]
    
def bonus(level, kind, relics, ascensions):
    factor = 40**(level - 1)
    multiplier = 5 if kind in increased_multiplier else 1
    return factor*relics*ascensions*0.001*30*multiplier

total = 0
for relic in relics:
    b = bonus(**relic)
    total += b
    if b > 0:
        print(relic['kind'], b)
total
    

moonstone 52.92
sand 53.64
emerald 128.25
ancient 102.00000000000001
tektite 158.4
bismuth 54.480000000000004
amber 262.5
obsidian 481.20000000000005
lava 97.44000000000001
ice 135.9
amethyst 25.500000000000004
celestial 65.4
silicon 79.92
rhodonite 132.75
platinum 96.24000000000001
cosmic 65.55
exotic 27.6
void 41.67
mythical 78.3
benitoite 307.49999999999994
gold 81.36
leaves 181.98
kyanite 53.099999999999994
ruby 207.60000000000002


2971.2000000000003

In [10]:
ascension_shard_de_cost = 100
fusion_de_cost = 50
de_essence_cost = 6
hours_per_witch = 146 / 60 / 60
base_crit_rate = 0.3463

base_property_bonuses = {
    'wem': 10**-8.5,
    'crit': 10**-10,
}


leaf_type_to_rarity = dict(ancient=16, sacred=17, biotite=18, malachite=19, hematite=20)
leaf_types = leaf_type_to_rarity.keys()
rarity_to_leaf_type = { rarity: leaf_type for leaf_type, rarity in leaf_type_to_rarity.items() }
max_rarity = max(rarity_to_leaf_type)


base_rarity = 14

leaf_type_to_fusion_cost = dict(sacred=5, biotite=7, malachite=11, hematite=19)

@cache
def total_fusion_cost(leaf_type: str) -> int:
    if leaf_type_to_rarity[leaf_type] <= leaf_type_to_rarity['ancient']:
        return 0
    
    previous_leaf_type = rarity_to_leaf_type[leaf_type_to_rarity[leaf_type] - 1]
    
    return total_fusion_cost(previous_leaf_type) * 6 + leaf_type_to_fusion_cost[leaf_type]

@cache
def leaf_bonus(leaf_type: str) -> int:
    return (leaf_type_to_rarity[leaf_type] - base_rarity) * 128

@cache
def ascension_cost(leaf_type: str, new_ascend_level: int) -> int:
    rarity = leaf_type_to_rarity[leaf_type]
    if rarity <= leaf_type_to_rarity['ancient']:
        return 0
    
    return (2 + math.ceil(1.5 ** (rarity - base_rarity))) * new_ascend_level

@cache
def total_ascension_cost(leaf_type: str, ascend_level: int) -> int:
    return sum(ascension_cost(leaf_type, level) for level in range(1, ascend_level + 1))

@dataclass(frozen=True)
class Leaf:
    leaf_type: str
    ascend_level: int = 10
    level: int = 30
    rng_quality: int = 100
    craft_quality: int = 100
    
    # dictionary of property names to the number of shards on that property
    properties: frozendict[str, int] = frozendict(wem=10, crit=7)
    
    def __post_init__(self):
        if self.leaf_type not in leaf_type_to_rarity:
            raise ValueError('leaf_type')
        if self.ascend_level not in range(0, 10 + 1):
            raise ValueError('ascend_level')
        if self.level not in range(0, 30 + 1):
            raise ValueError('level')
        if self.rng_quality not in range(1, 100 + 1):
            raise ValueError('rng_quality')
        if self.craft_quality not in range(1, 100 + 1):
            raise ValueError('craft_quality')
        
        for name, shards in self.properties.items():
            if name not in base_property_bonuses:
                raise ValueError('property.name')
            if shards not in range(0, 10 + 1):
                raise ValueError('property.shards')
            
        if sum(self.properties.values()) not in range(0, 17 + 1):
            raise ValueError('total_shards')
            
    @property
    def rarity(self) -> int:
        return leaf_type_to_rarity[self.leaf_type]
    
    def shards(self, property_name: str) -> int:
        return self.properties[property_name]

    def property_bonus(self, property_name: str) -> float:
        if property_name not in self.properties:
            return 0
        
        ascend_level = self.ascend_level

        if ascend_level == 0:
            ascend_level = -1

        return base_property_bonuses[property_name] * \
            (((self.ascend_level + 1)*30*2 + self.level)/5 + 1) * \
            leaf_bonus(self.leaf_type)**1.6 * \
            (1 + self.shards(property_name) * 3) * \
            (self.rng_quality + self.craft_quality) / 20

@cache
def de_cost(old_leaves: list[Leaf], new_leaves: list[Leaf]) -> float:
    fusion_shards = sum(total_fusion_cost(leaf.leaf_type) for leaf in new_leaves) - sum(total_fusion_cost(leaf.leaf_type) for leaf in old_leaves)
    
    def key(leaf: Leaf):
        return [leaf.rarity, leaf.ascend_level]
    
    sorted_old_leaves = sorted(old_leaves, key=key)
    sorted_new_leaves = sorted(new_leaves, key=key)
    
    ascension_shards = 0
    
    for leaf in reversed(sorted_new_leaves):
        while len(sorted_old_leaves) > 0 and key(sorted_old_leaves[-1]) > key(leaf):
            sorted_old_leaves.pop()
            
        if len(sorted_old_leaves) > 0 and leaf.rarity == sorted_old_leaves[-1].rarity:
            old_leaf = sorted_old_leaves[-1]
            ascension_shards += total_ascension_cost(leaf.leaf_type, leaf.ascend_level) - total_ascension_cost(old_leaf.leaf_type, old_leaf.ascend_level)
            sorted_old_leaves.pop()
        else:
            ascension_shards += total_ascension_cost(leaf.leaf_type, leaf.ascend_level)
            
    return (ascension_shards * ascension_shard_de_cost + fusion_shards * fusion_de_cost)

@cache
def total_essence_cost(old_leaves: list[Leaf], new_leaves: list[Leaf]) -> float:
    return de_cost(old_leaves, new_leaves) * de_essence_cost

def essence_per_witch(leaves: tuple[Leaf]) -> float:
    total_wem = 1 + sum(leaf.property_bonus('wem') for leaf in leaves)
    return total_wem * 17.9

def crit_rate(leaves: tuple[Leaf], base=base_crit_rate) -> float:
    return base + sum(leaf.property_bonus('crit') for leaf in leaves)

def leaves_factor(leaves: tuple[Leaf]):
    return essence_per_witch(leaves) * (1 + crit_rate(leaves))**2

def total_time_cost_through(calc_leaves: tuple[Leaf], old_leaves: tuple[Leaf], new_leaves: tuple[Leaf]):
    return total_essence_cost(old_leaves, new_leaves) * hours_per_witch / leaves_factor(calc_leaves)

def total_time_cost_with(old_leaves: tuple[Leaf], new_leaves: tuple[Leaf], essence_per_witch, crit_rate):
    return total_essence_cost(old_leaves, new_leaves) / essence_per_witch * hours_per_witch / (1 + crit_rate)**2

def total_time_cost(old_leaves: tuple[Leaf], new_leaves: tuple[Leaf]) -> float:
    return total_time_cost_through(old_leaves, old_leaves, new_leaves)
    
total_ascension_cost('hematite', 10)
de_cost((Leaf(leaf_type='ancient'),), (Leaf(leaf_type='hematite'),))
total_time_cost((Leaf(leaf_type='ancient'),) * 8, (Leaf(leaf_type='hematite'),) * 8)
leaves_factor((Leaf(leaf_type='ancient'),) * 8)
Leaf(leaf_type='ancient').property_bonus('wem')

# second_best_set = (Leaf(leaf_type='hematite', ascend_level=9),) + (Leaf(leaf_type='hematite'),) * 7
# best_set = (Leaf(leaf_type='hematite'),) * 8
# start_set = (Leaf(leaf_type='ancient'),) * 8
# total_time_cost(second_best_set, best_set)

0.9717631775806307

In [4]:


def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

def a_star(start, goal, neighbors, heuristic, print_best=True, debug=False):
    open_set = heapdict()
    open_set[start] = heuristic(start, goal)
    came_from = {}
    score = defaultdict(lambda: math.inf)
    score[start] = 0
    i = 0
    
    while len(open_set) > 0:
        if debug or (print_best and i % 1000 == 0):
            best_so_far, best_score = open_set.peekitem()
            print(len(open_set), f'best = {score[best_so_far]:.2f} + {best_score - score[best_so_far]:.2f} = {best_score:.3f}')
        i += 1
        
        current, current_score = open_set.popitem()
        
        if debug:
            print(current, current_score)
        
        if current == goal:
            return reconstruct_path(came_from, current), current_score
        
        for neighbor, distance in neighbors(current):
            tentative_score = score[current] + distance
            # print(score[current], distance(current, neighbor), tentative_score)
            if tentative_score < score[neighbor]:
                came_from[neighbor] = current
                score[neighbor] = tentative_score
                open_set[neighbor] = tentative_score + heuristic(neighbor, goal)

In [5]:
def leaf_set_neighbors(leaf_set):
    # let us assume that the leaf set is sorted
    # print(leaf_set)
    neighbors = []
    epw = essence_per_witch(leaf_set)
    cr = crit_rate(leaf_set)
    for i, leaf in enumerate(leaf_set):
        if i < len(leaf_set) - 1 and leaf == leaf_set[i + 1]:
            continue
        for new_rarity in range(leaf.rarity, max_rarity + 1):
            for new_ascend_level in range(leaf.ascend_level + 1 if leaf.rarity == new_rarity else 0, 10 + 1):
                new_leaf = replace(leaf, leaf_type=rarity_to_leaf_type[new_rarity], ascend_level=new_ascend_level)
                neighbor = [*leaf_set[:i], *leaf_set[i+1:], new_leaf]
                neighbor.sort(key=lambda leaf: [leaf.rarity, leaf.ascend_level])
                neighbors.append((tuple(neighbor), total_time_cost_with((leaf,), (new_leaf,), essence_per_witch=epw, crit_rate=cr)))
    return neighbors

def create_base_heuristic():
    epw = essence_per_witch(second_best_set)
    cr = crit_rate(second_best_set)
    def base_heuristic(begin, end):
        return total_time_cost_with(begin, end, essence_per_witch=epw, crit_rate=cr)
    return base_heuristic

base_heuristic = create_base_heuristic()

def create_heuristic_1(end):
    table = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))
    
    def heuristic(begin, _):
        return max(
            table[sum(1 if leaf.rarity < max_rarity or leaf.ascend_level < 10 else 0 for leaf in begin)]
                [max(leaf.rarity for leaf in begin)]
                [max(leaf.ascend_level for leaf in begin)],
            base_heuristic(begin, end)
        )
    
    for number_fully_upgraded in range(8, -1, -1):
        for rarity in range(max_rarity, leaf_type_to_rarity['ancient'] - 1, -1):
            for ascend_level in range(10 if rarity < max_rarity else 9, -1 if rarity > leaf_type_to_rarity['ancient'] else 9, -1):
                leaves = (Leaf(leaf_type=rarity_to_leaf_type[rarity], ascend_level=ascend_level),)*(8-number_fully_upgraded) + \
                    (Leaf(leaf_type='hematite'),)*number_fully_upgraded
                _, time = a_star(leaves, end, leaf_set_neighbors, heuristic, print_best=False)
                print(number_fully_upgraded, rarity_to_leaf_type[rarity], ascend_level, time)
                table[rarity][ascend_level] = time
    
    return heuristic

def create_possible_tests(depth, max_rarity=max_rarity, max_ascend_level=11, max_count=8):
    if max_count == 0:
        yield tuple()
        return
    
    if depth == 0:
        yield (Leaf(leaf_type=rarity_to_leaf_type[max_rarity if max_ascend_level > 0 else max_rarity - 1], ascend_level=max_ascend_level - 1 if max_ascend_level > 0 else 10),) * max_count
        return
    
    for rarity in range(max_rarity, leaf_type_to_rarity['ancient'] - 1, -1):
        for ascend_level in range(10 if rarity < max_rarity else max_ascend_level - 1, -1 if rarity > leaf_type_to_rarity['ancient'] else 9, -1):
            for count in range(max_count, max_count - 1 if rarity == leaf_type_to_rarity['ancient'] else 0, -1):
                for rest in create_possible_tests(depth=depth-1, max_rarity=rarity, max_ascend_level=ascend_level, max_count=max_count-count):
                    yield rest + (Leaf(leaf_type=rarity_to_leaf_type[rarity], ascend_level=ascend_level),) * count
    

def create_heuristic_2(end, depth=1):
    table = defaultdict(lambda: 0)
    
    def key(leaves):
        key = []
        current_depth = 0
        previous = None
        count = 0
        append_end = True
        
        for i, leaf in enumerate(reversed(leaves)):
            if leaf == previous:
                count += 1
            else:
                if previous is not None:
                    key.append((previous.rarity, previous.ascend_level, count))
                    current_depth += 1
                previous = leaf
                count = 1
            
            if current_depth >= depth:
                append_end = False
                break
                
        if append_end and not ((previous.rarity == max_rarity and previous.ascend_level == 10)
                               if len(key) == 0
                               else (previous.rarity == key[-1][0] and previous.ascend_level == key[-1][1] - 1) or (previous.rarity == key[-1][0] - 1 and previous.ascend_level == 10 and key[-1][1] == 0)):
            key.append((previous.rarity, previous.ascend_level, count))
        
        return tuple(key)
    
    def heuristic(begin, _):
        best = base_heuristic(begin, end)
        lookup = key(begin)
        
        for i in range(1, len(lookup)):
            best = max(best, table[lookup[:i]])
                       
        return best
    
    to_test = list(create_possible_tests(depth))
    all_keys = [key(leaves) for leaves in to_test]
    
    while len(to_test) > 0:
        to_test.sort(key=lambda begin: -heuristic(begin, end))
        leaves = to_test.pop()
        save_key = key(leaves)
        _, time = a_star(leaves, end, leaf_set_neighbors, heuristic, print_best=False)
        print(save_key, time)
        table[save_key] = time
    
    for leaves in sorted(create_possible_tests(depth), key=lambda begin: previous_heuristic(begin, end)):
        save_key = key(leaves)
        # print(leaves, save_key)
        path, time = a_star(leaves, end, leaf_set_neighbors, heuristic, print_best=False)
        print(save_key, time)
        for key_to_consider in all_keys:
            if len(key_to_consider) >= len(save_key) and all(zip(save_key, key_to_consider), lambda a, b: all(zip(a,b), lambda a1, b1: b1 <= a1)):
                table[key_to_consider] = max(time, table[key_to_consider])
    
    return reversed_min_heuristic

# optimal_path = a_star(start_set, best_set, leaf_set_neighbors, create_heuristic_2(best_set, depth=3))

NameError: name 'second_best_set' is not defined

In [ ]:
def leaf_set_neighbors_2(leaf_set):
    # let us assume that the leaf set is sorted
    # print(leaf_set)
    neighbors = []
    epw = essence_per_witch(leaf_set)
    cr = crit_rate(leaf_set)
    for i, leaf in enumerate(leaf_set):
        if i < len(leaf_set) - 1 and leaf == leaf_set[i + 1]:
            continue
        for new_rarity in range(16, leaf.rarity + 1):
            for new_ascend_level in range(10 if new_rarity == 16 else 0, 10 + 1 if new_rarity < leaf.rarity else leaf.ascend_level):
                new_leaf = replace(leaf, leaf_type=rarity_to_leaf_type[new_rarity], ascend_level=new_ascend_level)
                neighbor = [*leaf_set[:i], *leaf_set[i+1:], new_leaf]
                neighbor.sort(key=lambda leaf: [leaf.rarity, leaf.ascend_level])
                neighbors.append((tuple(neighbor), total_time_cost_with((new_leaf,), (leaf,), essence_per_witch=epw, crit_rate=cr)))
    return neighbors

def heuristic_3(begin, end):
    return total_time_cost_through(begin, end, begin)

ancient_leaf = Leaf(leaf_type='ancient')
seven_ancients = (ancient_leaf,) * 7

@cache
def heuristic_4_leaf(biggest_leaf):
    if biggest_leaf == ancient_leaf:
        return 0
    
    _, cost = a_star(seven_ancients + (biggest_leaf,), (ancient_leaf,) * 8, leaf_set_neighbors_2, heuristic_3, debug=True)
    
    return cost

def heuristic_4(begin, end):
    return max(heuristic_4_leaf(begin[-1]), heuristic_3(begin, end))
    

optimal_path = a_star(best_set, start_set, leaf_set_neighbors_2, heuristic_3)

In [ ]:
optimal_path

In [11]:
10**-8.5

3.1622776601683795e-09